## Big Data

When you say Big data we are looking at data sets so large and so complex that it's not possible to process them on one computer. 
They are impossible to process using the traditional databases and tools that have existed for the last 40 years in computer science. 

Hadoop, distributed processing, map-reduce programming are some of the words that pop into someone's mind when they think of big data with an assumption that its the ideal choice when dealing with large datasets. 
Lets look at a simple example which is discussed in Database Analytics course. 
We will compare the time taken by a traditional PostgreSQL database and Apache drill + Hadoop 

------

### Pulling a few rows from a large table

#### PostgreSQL using the psycopg interface

In [ ]:
import time
import psycopg2
import numpy as np
import pandas as pd

# The typical Connection String format for PostgreSQL
connect_str = "dbname='twitter' user='dsa_ro_user' host='dbase.dsa.missouri.edu'password='readonly'"

with psycopg2.connect(connect_str) as conn:

    # Accumulate total time and list
    total_time = 0.0
    times = []
    
    cursor = conn.cursor()

    ############### START Timing Iterations
    for n in range(0,35):
        
        # Start the Timer
        start = time.perf_counter()

        # This is a very basic query, 
        # fetch any 500 rows from a table of over 100 million
        cursor.execute("SELECT * FROM twitter.hashtag LIMIT 500")

        # This pulls all the result rows in the Python Memory
        cols = cursor.fetchall()

        end = time.perf_counter()

        # Accumulate and list
        total_time += (end - start)
        times.append(end - start)

    ############### END Timing Iterations
        
    # How long did this look up take, typically?
    print('Time Fetch Rows   ')
    print('------------------')
    print("Median Fetch Time: {0:.5f}".format(np.median(times)))
    print("Total Fetch Time: {0:.5f}".format(np.sum(times)))



### Hadoop + Drill using the PyDrill interface

In [ ]:
import time
from pydrill.client import PyDrill

drill = PyDrill(host='rvip.cgi.missouri.edu', port=8047)

if not drill.is_active():
    raise ImproperlyConfigured('Please run Drill first')

############### START Timing Iterations
for n in range(0,35):

    # Start the Timer
    start = time.perf_counter()

    # This is a very basic query, 
    # fetch any 500 rows from a table of over 100 million
    # This pulls all the result rows in the Python Memory
    rows = drill.query("SELECT * FROM  dfs.datasets.`twitter/hashtag.json` LIMIT 500")

    end = time.perf_counter()

    # Accumulate and list
    total_time += (end - start)
    times.append(end - start)

############### END Timing Iterations

# How long did this look up take, typically?
print('Time Fetch Rows   ')
print('------------------')
print("Median Fetch Time: {0:.5f}".format(np.median(times)))
print("Total Fetch Time: {0:.5f}".format(np.sum(times)))



Talking about infrastructure, 

#### The 16-core PostgreSQL server is out performing the 280-core Hadoop cluster?

##### Yes, and to add to that ... 
<span style="background:yellow">
PostgreSQL does not parallelize within-query execution.</span>
**Those PG times are from a single core!**

This is an important concept that is lost on many people that get caught up in the hype of "Big Data Ecosystems".

** Big Data $\neq$ High-Performance **

All the big data ecosystems built on top of Hadoop will lose this test, 
becase the tasks is aligned against the assumptions and technical decisions that went in to developing Hadoop.

**The technical why?**

This particular query gets no acceleration from "divide and conquer" strategy.
Recall that Hadoop does not support local cache of file blocks, partially due to the massive _block size_ (64MB) versus a tradition block size (4KB - 8KB).
PostgreSQL on the otherhand was designed and built using *systems programming* techniques, 
leveraging _interprocess communication_ technology, specifically Shared Memory Segments in this case.
The PostgreSQL DB caches each _page file_ (the on-disk partion blocks of table rows) it reads from disk into shared memory segments (RAM).
Therefore, running the same query 100 or a million times in fast succession on static data has 
only one single filesystem read phase.
After the first run of the SQL, PostgreSQL knows that it does not need all the data, 
only a small set (in this case) of page files.
These are cached in the Shared Memory Allocation, 
and therefore accessible to any next processes or the same process that needs that data or executes that same query.

In contrast, all 10 nodes of the Hadoop system must read the data each time.


##### Large Distributed Processing
So, Hadoop is designed for large distributed data processing that addresses every file in the database. 
And that type of processing takes time. 
For tasks like running end-of-day reports to review daily transactions, scanning historical data where performance/speed is not critical Hadoop is ideal.

On the other hand, in cases where organizations rely on time-sensitive data analysis,
a traditional database is the better fit. 
Hadoop does so well when it is needed to analyze large unstructured datasets. 
Traditional databases are well equipped to analyze smaller data sets in real time.

Because of the size and complexity nature of Big Data it is hard to capture, store, copy, share, search, analyze, visualize, delete etc.
There are many challenges dealing with big data. 
But it lets us gain insights and competitive edge by leveraging mountains of valuable data.
Big Data represents the information assets that are characterized by high volume, 
high velocity of data of different varierties. 
By applying different computational and analytical methods, 
this vast data gets transformed into something worthwhile for society and businesses. 


For example, internet of things, social networks, heavy machinery like telescopes, 
Medical data, genomics all are producing huge amounts of information. 
What do we do with this data? <span style="background:yellow">Analytics</span>. 
What we're looking for is, predictive analytics, so you can do forecasts and projections. 
Find connections and correlations to discover hidden patterns or maybe discover a new physics law 
or find a gene pattern which is responsible for a certain disease.
So where's the future?
We cannot do predictive analytics on vast data on a single computer.
We need to put that into a cloud to do that. 